In [1]:
# =============================================================
# 🧠 PATTERN REFERENCE MAP GENERATOR for CNN/DenseNet Training
# Author: ChatGPT Pattern Lab 2025
# Purpose: Create geometric bite pattern reference maps
# Location: /content/drive/MyDrive/Dataset/synthetic_pattern_ref/
# =============================================================

from PIL import Image, ImageDraw, ImageFilter
import numpy as np
import cv2, os
from google.colab import drive
from tqdm import tqdm

# --- Mount Drive ---
drive.mount('/content/drive')

# --- Setup folders ---
base_dir = "/content/drive/MyDrive/Dataset/synthetic_pattern_ref"
os.makedirs(base_dir, exist_ok=True)
ant_dir = os.path.join(base_dir, "ants_pattern_ref")
bed_dir = os.path.join(base_dir, "bedbugs_pattern_ref")
os.makedirs(ant_dir, exist_ok=True)
os.makedirs(bed_dir, exist_ok=True)

# --- Utility: generate subtle noise texture for realism ---
def perlin_texture(size=(224,224), scale=10):
    noise = np.random.rand(size[1]//scale, size[0]//scale)
    noise = cv2.resize(noise, size, interpolation=cv2.INTER_CUBIC)
    noise = (noise - noise.min()) / (noise.max() - noise.min())
    return noise

def make_highcontrast_spot(radius, color, blur=1):
    size = int(radius * 4)
    spot = Image.new("RGBA", (size, size), (0,0,0,0))
    draw = ImageDraw.Draw(spot)
    cx, cy = size//2, size//2
    for r in range(int(radius), 0, -1):
        alpha = int(255 * (r / radius)**1.3)
        fill = (color[0], color[1], color[2], alpha)
        draw.ellipse((cx-r, cy-r, cx+r, cy+r), fill=fill)
    spot = spot.filter(ImageFilter.GaussianBlur(blur))
    return spot

# --- Pattern generator (reference-style, geometric focus) ---
def generate_reference_pattern(size=(224,224), bite_type='ant'):
    W, H = size
    pattern = Image.new("RGBA", (W, H), (255,255,255,0))  # transparent background

    if bite_type == 'ant':
        clusters = np.random.randint(2, 5)
        for _ in range(clusters):
            cx, cy = np.random.randint(40,W-40), np.random.randint(40,H-40)
            for _ in range(np.random.randint(4,10)):
                r = np.random.randint(6,18)
                redness = np.random.randint(150,230)
                color = (redness, np.random.randint(0,40), np.random.randint(30,60), 255)
                spot = make_highcontrast_spot(r, color, blur=np.random.uniform(0.8,2))
                x, y = cx+np.random.randint(-40,40), cy+np.random.randint(-40,40)
                pattern.paste(spot, (x-spot.width//2, y-spot.height//2), spot)
    else:
        lines = np.random.randint(1,3)
        for _ in range(lines):
            start_x, start_y = np.random.randint(40,W-80), np.random.randint(60,H-60)
            spacing = np.random.randint(16,24)
            direction = np.random.uniform(-0.25, 0.25)
            for i in range(np.random.randint(3,6)):
                x = int(start_x + i*spacing*np.cos(direction))
                y = int(start_y + i*spacing*np.sin(direction))
                r = np.random.randint(7,14)
                redness = np.random.randint(170,250)
                color = (redness, np.random.randint(10,50), np.random.randint(40,80), 255)
                spot = make_highcontrast_spot(r, color, blur=np.random.uniform(0.8,1.8))
                pattern.paste(spot, (x-spot.width//2, y-spot.height//2), spot)

    # Apply soft Perlin noise to mimic textured reference
    arr = np.array(pattern).astype(np.float32)
    perlin = perlin_texture((W,H), scale=14)
    for c in range(3):
        arr[...,c] = np.clip(arr[...,c] * (0.85 + 0.25*perlin), 0, 255)
    return Image.fromarray(arr.astype('uint8'), 'RGBA')

# --- Generate 200 reference patterns ---
num_ants = 100
num_bedbugs = 100

print("🟢 Generating ANT pattern references...")
for i in tqdm(range(num_ants)):
    img = generate_reference_pattern(bite_type='ant')
    img.save(os.path.join(ant_dir, f"ant_ref_{i:03d}.png"))

print("🔵 Generating BEDBUG pattern references...")
for i in tqdm(range(num_bedbugs)):
    img = generate_reference_pattern(bite_type='bedbug')
    img.save(os.path.join(bed_dir, f"bedbug_ref_{i:03d}.png"))

print("\n✅ DONE! Pattern reference maps saved to:")
print(base_dir)


Mounted at /content/drive
🟢 Generating ANT pattern references...


  0%|          | 0/100 [00:00<?, ?it/s]/tmp/ipython-input-3529579456.py:80: DeprecationWarning: 'mode' parameter is deprecated and will be removed in Pillow 13 (2026-10-15)
  return Image.fromarray(arr.astype('uint8'), 'RGBA')
100%|██████████| 100/100 [00:03<00:00, 29.71it/s]


🔵 Generating BEDBUG pattern references...


100%|██████████| 100/100 [00:02<00:00, 40.60it/s]


✅ DONE! Pattern reference maps saved to:
/content/drive/MyDrive/Dataset/synthetic_pattern_ref
